In [2]:
import pandas as pd

df = pd.read_csv("../data/parliament_data.csv")
df.head()

,session_pdf_url,session_year,session_date,session_pdf_text_path,session_pdf_download_status,proposal_name_from_session,proposal_gov_link,voting_details_json,session_parse_status,proposal_authors_json,...,proposal_summary_fiscal_impact,proposal_summary_colloquial,proposal_category,proposal_summarize_status,proposal_approval_status,proposal_short_title,proposal_proposing_party,overall_status,last_error_message,last_processed_timestamp
0,https://app.parlamento.pt/WebUtils/docs/doc.pd...,2020.0,2020-04-02,data/session_pdfs/2020_XIV_1_44_2020-04-02_202...,Success,NaN,NaN,NaN,LLM Parsed - No Propostas Encontradas,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed (No Propostas),NaN,2025-06-14T22:20:53.561405
1,https://app.parlamento.pt/WebUtils/docs/doc.pd...,2020.0,2020-02-06,data/session_pdfs/2020_XIV_1_26_2020-02-06_202...,Success,Proposta de Lei n.º 4/XIV/1.ª (GOV),https://www.parlamento.pt/ActividadeParlamenta...,NaN,Success,NaN,...,A proposta prevê um aumento dos gastos do gove...,Este é o plano do governo para os próximos qua...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",Success,1.0,Grandes Opções do Plano 2020-2023,Governo,Success,NaN,2025-06-14T22:45:04.324594
2,https://app.parlamento.pt/WebUtils/docs/doc.pd...,2020.0,2020-04-02,data/session_pdfs/2020_XIV_1_44_2020-04-02_202...,Success,Proposta de Lei n.º 18/XIV/1.ª,https://www.parlamento.pt/ActividadeParlamenta...,NaN,Success,NaN,...,Não há impacto fiscal direto mencionado na pro...,"Em termos simples, por causa da pandemia de CO...","[0, 9]",Success,1.0,Suspensão de Contratos de Arrendamento devido ...,Governo,Success,NaN,2025-06-14T22:21:47.282858
3,https://app.parlamento.pt/WebUtils/docs/doc.pd...,2020.0,2020-04-02,data/session_pdfs/2020_XIV_1_44_2020-04-02_202...,Success,Proposta de Lei n.º 20/XIV/1.ª,https://www.parlamento.pt/ActividadeParlamenta...,"{""PS"": {""Favor"": 108, ""Contra"": 0, ""Absten\u00...",Success,NaN,...,Esta proposta de lei permitirá aos municípios ...,"Basicamente, esta lei é para que as câmaras mu...","[0, 4, 5]",Success,1.0,Flexibilização financeira municipal para comba...,Governo,Success,NaN,2025-06-14T22:21:51.304886
4,https://app.parlamento.pt/WebUtils/docs/doc.pd...,2020.0,2020-04-02,data/session_pdfs/2020_XIV_1_44_2020-04-02_202...,Success,Proposta de Lei n.º 21/XIV/1.ª,https://www.parlamento.pt/ActividadeParlamenta...,NaN,Success,NaN,...,A proposta implicará um aumento dos gastos do ...,"Em termos simples, esta lei é para dar uma aju...","[0, 4, 5, 9]",Success,1.0,Medidas Excecionais para Rendas na Pandemia CO...,Governo,Success,NaN,2025-06-14T22:21:57.219208


In [3]:
df.columns

Index(['session_pdf_url', 'session_year', 'session_date',
       'session_pdf_text_path', 'session_pdf_download_status',
       'proposal_name_from_session', 'proposal_gov_link',
       'voting_details_json', 'session_parse_status', 'proposal_authors_json',
       'proposal_document_url', 'proposal_document_type',
       'proposal_document_local_path', 'proposal_doc_download_status',
       'proposal_details_scrape_status', 'proposal_summary_general',
       'proposal_summary_analysis', 'proposal_summary_fiscal_impact',
       'proposal_summary_colloquial', 'proposal_category',
       'proposal_summarize_status', 'proposal_approval_status',
       'proposal_short_title', 'proposal_proposing_party', 'overall_status',
       'last_error_message', 'last_processed_timestamp'],
      dtype='object')

In [11]:
df['proposal_document_url'].unique().shape, df['proposal_document_url'].nunique(), df['proposal_document_url'].shape

((4892,), 4891, (5633,))

In [14]:
# Step 1: Drop rows with NaN values in proposal_document_url
df_clean = df.dropna(subset=['proposal_document_url'])

# Step 2: For duplicates, prioritize rows with proposal_approval_status = 1.0
# First, sort by proposal_approval_status (descending) and session_date (descending)
df_clean['session_date'] = pd.to_datetime(df_clean['session_date'])
df_clean = df_clean.sort_values(['proposal_document_url', 'proposal_approval_status', 'session_date'], 
                                ascending=[True, False, False])

# Step 3: Keep the first occurrence after sorting (which will be the one with highest approval status and most recent date)
df_clean = df_clean.drop_duplicates(subset=['proposal_document_url'], keep='first')

print(f"Original shape: {df.shape}")
print(f"After cleaning: {df_clean.shape}")
print(f"Unique proposal_document_urls: {df_clean['proposal_document_url'].nunique()}")

Original shape: (5633, 27)
After cleaning: (4891, 27)
Unique proposal_document_urls: 4891


/var/folders/6s/st37x6x51cd3xjgrvd46grq80000gn/T/ipykernel_92949/3826263607.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['session_date'] = pd.to_datetime(df_clean['session_date'])


In [ ]:
df_clean.to_csv("../data/parliament_data.csv", index=False)